In [18]:
import pandas as pd
import os
import requests
import numpy as np 

print('Current working directory:', os.getcwd())

Current working directory: /home/prichter/Documents/find-a-bug/sandbox


In [9]:
LOG_DIR = os.path.join(os.getcwd(), '../scripts/log/')

# Running into errors uploading certain annotations to the MariaDB database. 
failures_df = []
for file_name in os.listdir(LOG_DIR):
    failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))
failures_df = pd.concat(failures_df)

/tmp/ipykernel_233074/888517050.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))
/tmp/ipykernel_233074/888517050.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))


In [12]:
print(len(failures_df.genome_id.unique()), 'genome IDs failed to upload.')
# Perhaps I should first make sure the files are actually present in the 

1100 genome IDs failed to upload.


In [17]:
failing_genome_ids = failures_df['genome_id'].unique()
np.savetxt('failing_genome_ids.txt', failing_genome_ids, fmt='%s')

In [35]:
for genome_id in failing_genome_ids:
    url = f'http://microbes.gps.caltech.edu:8000/get/proteins_r207?genome_id[eq]{genome_id}'
    result = requests.get(url).text 
    print(result)
    break
    

<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

